In [ ]:
import numpy as np
import sklearn
from scipy.linalg import khatri_rao
from sklearn.svm import LinearSVC
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge
import sys
import time

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:

def ReadDataFromFile(path):

  # Read data from file into a NumPy array
  data = np.loadtxt(path,dtype="int")

  # Extract features (first 32 columns) and targets (last column)
  features = data[:, :32]
  targets = data[:, 32]

  return features,targets

path1='/content/drive/MyDrive/CS771/Assign1/train.dat'
path2='/content/drive/MyDrive/CS771/Assign1/test.dat'
X_train,y_train = ReadDataFromFile(path1)
X_test,y_test = ReadDataFromFile(path2)

In [ ]:
X_train[0]

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
d_train = 1-2*X_train  # for every training challenge bit, calculate (1-2ci)
d_test = 1-2*X_test    # for every testing challenge bit, calculate (1-2ci)
print(d_train.shape)
print(d_test.shape)


In [ ]:
print(d_train[0])
print(d_test[0])

In [ ]:
# X = [[1 for _ in range(d.shape[0]) ] for _ in range(d.shape[1])]
# for i in range(d.shape[0]):
#   for j in range(d.shape[1]-1,-1,-1):
#     if(j==31):
#       X[i][j]=d[i][j]
#     else:
#       X[i][j] = X[i][j-1] * d[i][j]

In [ ]:
def makex(d):
  return np.fliplr(np.cumprod(np.fliplr(d), axis=1))
  # # # Initialize X as an empty list
  # # x = []

  # # # Loop through each row of d
  # # for row in d:
  # #     # Initialize temp as an empty list for each row
  # #     temp = []
  # #     # Loop through the first 32 elements of each row
  # #     for j in range(32):
  # #         # Calculate the product of the first 32 elements of each row using numpy's prod function
  # #         t = np.prod(row[j:j+32])
  # #         # Append the result to temp
  # #         temp.append(t)
  # #     # Append temp to X
  # #     x.append(temp)

  # # Convert X to a numpy array
  # x = np.array(x)
  # return x

x_train=makex(d_train)
x_test=makex(d_test)

In [ ]:
print(x_train.shape)
print(x_test.shape)
print(x_train[0].shape)
print(x_test[0].shape)

In [ ]:
x_train[0]

In [ ]:


def makenew(X):

  X_new=[]
  for i in range(X.shape[0]):
    temp=[]
    for j in range(32):
      for k in range(j,32):
        if(j==k): temp.append(X[i][j])
        else: temp.append(X[i][j]*X[i][k])
    X_new.append(temp)
  X_new=np.array(X_new)
  return X_new

X_new_train = makenew(x_train)
X_new_test = makenew(x_test)

In [ ]:
print(X_new_train.shape)
# print(X_new_test.shape)

In [ ]:
y_train=np.ravel(y_train)
y_test=np.ravel(y_test)

In [ ]:
model = LinearSVC()

In [ ]:
model.fit(X_new_train, y_train)

In [ ]:
predictions =  model.predict(X_new_test)
accuracy_score(y_test, predictions)

In [ ]:
model2=LogisticRegression()

In [ ]:
model2.fit(X_new_train,y_train)

In [ ]:
predictions2 = model2.predict(X_new_test)
accuracy_score(y_test, predictions2)

In [ ]:
#comparisions

#changing loss hyperparameter
loss = ['squared_hinge', 'hinge']
time_taken_loss = []
accuracy_score_loss = []
for (i,l) in enumerate(loss):
  print(i, l)
  if i:
    model  = LinearSVC(penalty = 'l2', loss = l)
  tic = time.perf_counter()
  model.fit(X_new_train, y_train)
  toc = time.perf_counter()
  time_taken = toc - tic
  time_taken_loss.append(time_taken)
  predictions =  model.predict(X_new_test)
  acc_score = accuracy_score(y_test, predictions)
  accuracy_score_loss.append(acc_score)


In [ ]:
print(time_taken_loss)
print(accuracy_score_loss)

In [ ]:
high = 10.0
med = 1.0
low = 0.1
#setting low, medium and high values of C
time_taken_c_linearSVC = []
accuracy_score_c_linearSVC = []

time_taken_c_LogisticRegression = []
accuracy_score_c_LogisticRegression = []

c_val = [pow(10,x) for x in range(-6,7)]

for c in c_val:
  model1 = LinearSVC(C=c)

  tic = time.perf_counter()
  model1.fit(X_new_train, y_train)
  toc = time.perf_counter()
  time_taken = toc - tic
  time_taken_c_linearSVC.append(time_taken)
  predictions =  model1.predict(X_new_test)
  acc_score = accuracy_score(y_test, predictions)
  accuracy_score_c_linearSVC.append(acc_score)

for c in c_val:
  model2=LogisticRegression( C = c)

  tic = time.perf_counter()
  model2.fit(X_new_train, y_train)
  toc = time.perf_counter()
  time_taken = toc - tic
  time_taken_c_LogisticRegression.append(time_taken)
  predictions =  model2.predict(X_new_test)
  acc_score = accuracy_score(y_test, predictions)
  accuracy_score_c_LogisticRegression.append(acc_score)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Define colors for each point
colors = plt.cm.viridis(np.linspace(0, 1, len(c_val)))  # Use viridis colormap

# Create a figure and subplots with specified size and space between them
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6), gridspec_kw={'wspace': 0.5})  # Adjust the width and height as needed

# Plot data on the first subplot with log scale x-axis
ax1.plot(c_val, time_taken_c_linearSVC, color='blue', linestyle='-')
for i in range(len(c_val)):
    ax1.scatter(c_val[i], time_taken_c_linearSVC[i], color=colors[i], label=f'C: {c_val[i]:.1e} Time: {round(time_taken_c_linearSVC[i], 2)}sec')
ax1.set_xlabel('C Value (log scale)')
ax1.set_ylabel('Time Taken (seconds)')
ax1.set_title('Time Taken vs. C Value (linearSVC)')
ax1.set_xscale('log')
ax1.grid(True)  # Show gridlines
ax1.legend(loc='lower right',fontsize='small')

# Plot data on the second subplot with log scale x-axis
ax2.plot(c_val, accuracy_score_c_linearSVC, color='blue', linestyle='-')
for i in range(len(c_val)):
    ax2.scatter(c_val[i], accuracy_score_c_linearSVC[i], color=colors[i], label=f'C: {c_val[i]:.1e}, Accuracy: {round(accuracy_score_c_linearSVC[i]*100,2)}%')
ax2.set_xlabel('C Value (log scale)')
ax2.set_ylabel('Accuracy Score')
ax2.set_title('Accuracy Score vs. C Value (linearSVC)')
ax2.set_xscale('log')
ax2.grid(True)  # Show gridlines
ax2.legend(loc='lower right',fontsize='small')

# Adjust layout to prevent overlap
plt.tight_layout()

# Show the plot
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Define colors for each point
colors = plt.cm.viridis(np.linspace(0, 1, len(c_val)))  # Use viridis colormap

# Create a figure and subplots with specified size and space between them
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6), gridspec_kw={'wspace': 0.5})  # Adjust the width and height as needed

# Plot data on the first subplot with log scale x-axis
ax1.plot(c_val, time_taken_c_LogisticRegression, color='blue', linestyle='-')
for i in range(len(c_val)):
    ax1.scatter(c_val[i], time_taken_c_LogisticRegression[i], color=colors[i], label=f'C: {c_val[i]:.1e} Time: {round(time_taken_c_LogisticRegression[i], 2)}sec')
ax1.set_xlabel('C Value (log scale)')
ax1.set_ylabel('Time Taken (seconds)')
ax1.set_title('Time Taken vs. C Value (LogisticRegression)')
ax1.set_xscale('log')
ax1.grid(True)  # Show gridlines
ax1.legend(loc='upper left',fontsize='small')

# Plot data on the second subplot with log scale x-axis
ax2.plot(c_val, accuracy_score_c_LogisticRegression, color='blue', linestyle='-')
for i in range(len(c_val)):
    ax2.scatter(c_val[i], accuracy_score_c_LogisticRegression[i], color=colors[i], label=f'C: {c_val[i]:.1e}, Accuracy: {round(accuracy_score_c_LogisticRegression[i]*100,2)}%')
ax2.set_xlabel('C Value (log scale)')
ax2.set_ylabel('Accuracy Score')
ax2.set_title('Accuracy Score vs. C Value (LogisticRegression)')
ax2.set_xscale('log')
ax2.grid(True)  # Show gridlines
ax2.legend(loc='lower right',fontsize='small')

# Adjust layout to prevent overlap
plt.tight_layout()

# Show the plot
plt.show()


In [ ]:
print(time_taken_c_linearSVC)
print(accuracy_score_c_linearSVC)
print(time_taken_c_LogisticRegression)
print(accuracy_score_c_LogisticRegression)

In [ ]:
#changing tol value :

tol_val = [pow(10,x) for x in range(-7,5)]
time_taken_tol_linearSVC = []
accuracy_score_tol_linearSVC = []
time_taken_tol_LogisticRegression = []
accuracy_score_tol_LogisticRegression = []

for tol in tol_val:
  model1 = LinearSVC(tol = tol)

  tic = time.perf_counter()
  model1.fit(X_new_train, y_train)
  toc = time.perf_counter()
  time_taken = toc - tic
  time_taken_tol_linearSVC.append(time_taken)

  predictions =  model1.predict(X_new_test)
  acc_score = accuracy_score(y_test, predictions)
  accuracy_score_tol_linearSVC.append(acc_score)

for tol in tol_val:
  model2= LogisticRegression(tol = tol)

  tic = time.perf_counter()
  model2.fit(X_new_train, y_train)
  toc = time.perf_counter()
  time_taken = toc - tic
  time_taken_tol_LogisticRegression.append(time_taken)

  predictions =  model2.predict(X_new_test)
  acc_score = accuracy_score(y_test, predictions)
  accuracy_score_tol_LogisticRegression.append(acc_score)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Define colors for each point
colors = plt.cm.viridis(np.linspace(0, 1, len(tol_val)))  # Use viridis colormap

# Create a figure and subplots with specified size and space between them
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6), gridspec_kw={'wspace': 0.5})  # Adjust the width and height as needed

# Plot data on the first subplot with log scale x-axis
ax1.plot(tol_val, time_taken_tol_LogisticRegression, color='blue', linestyle='-')
for i in range(len(tol_val)):
    ax1.scatter(tol_val[i], time_taken_tol_LogisticRegression[i], color=colors[i], label=f'Tol: {tol_val[i]:.1e} Time: {round(time_taken_tol_LogisticRegression[i], 2)}sec')
ax1.set_xlabel('Tolerance Value (log scale)')
ax1.set_ylabel('Time Taken (seconds)')
ax1.set_title('Time Taken vs. Tolerance Value (LogisticRegression)')
ax1.set_xscale('log')
ax1.grid(True)  # Show gridlines
ax1.legend(loc='lower left',fontsize='small')

# Plot data on the second subplot with log scale x-axis
ax2.plot(tol_val, accuracy_score_tol_LogisticRegression, color='blue', linestyle='-')
for i in range(len(tol_val)):
    ax2.scatter(tol_val[i], accuracy_score_tol_LogisticRegression[i], color=colors[i], label=f'Tol: {tol_val[i]:.1e}, Accuracy: {round(accuracy_score_tol_LogisticRegression[i]*100,2)}%')
ax2.set_xlabel('Tolerance Value (log scale)')
ax2.set_ylabel('Accuracy Score')
ax2.set_title('Accuracy Score vs. Tolerance Value (LogisticRegression)')
ax2.set_xscale('log')
ax2.grid(True)  # Show gridlines
ax2.legend(loc='lower left',fontsize='small')

# Adjust layout to prevent overlap
plt.tight_layout()

# Show the plot
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Define colors for each point
colors = plt.cm.viridis(np.linspace(0, 1, len(tol_val)))  # Use viridis colormap

# Create a figure and subplots with specified size and space between them
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6), gridspec_kw={'wspace': 0.5})  # Adjust the width and height as needed

# Plot data on the first subplot with log scale x-axis
ax1.plot(tol_val, time_taken_tol_linearSVC, color='blue', linestyle='-')
for i in range(len(tol_val)):
    ax1.scatter(tol_val[i], time_taken_tol_linearSVC[i], color=colors[i], label=f'Tol: {tol_val[i]:.1e} Time: {round(time_taken_tol_linearSVC[i], 2)}sec')
ax1.set_xlabel('Tolerance Value (log scale)')
ax1.set_ylabel('Time Taken (seconds)')
ax1.set_title('Time Taken vs. Tolerance Value (LinearSVC)')
ax1.set_xscale('log')
ax1.grid(True)  # Show gridlines
ax1.legend(loc='lower left',fontsize='small')

# Plot data on the second subplot with log scale x-axis
ax2.plot(tol_val, accuracy_score_tol_linearSVC, color='blue', linestyle='-')
for i in range(len(tol_val)):
    ax2.scatter(tol_val[i], accuracy_score_tol_linearSVC[i], color=colors[i], label=f'Tol: {tol_val[i]:.1e}, Accuracy: {round(accuracy_score_tol_linearSVC[i]*100,2)}%')
ax2.set_xlabel('Tolerance Value (log scale)')
ax2.set_ylabel('Accuracy Score')
ax2.set_title('Accuracy Score vs. Tolerance Value (LinearSVC)')
ax2.set_xscale('log')
ax2.grid(True)  # Show gridlines
ax2.legend(loc='lower left',fontsize='small')

# Adjust layout to prevent overlap
plt.tight_layout()

# Show the plot
plt.show()


In [ ]:
# Create a figure and subplots
fig, (ax1, ax2) = plt.subplots(1, 2 , figsize=(12, 6) , gridspec_kw={'wspace': 0.5})

# Plot data on the first subplot
ax1.plot(tol_val, time_taken_tol_linearSVC)
ax1.set_xlabel('X axis')
ax1.set_ylabel('tols')
ax1.set_xscale('log')
ax1.set_title('Time for Linear SVC at different tols')
ax1.grid(True)

# Plot data on the second subplot
ax2.plot(tol_val, accuracy_score_tol_linearSVC)
ax2.set_xscale('log')
ax2.set_xlabel('tols')
ax2.set_ylabel('Accuracy')
ax2.set_title('Accuracy for Linear SVC at different tols')
ax2.grid(True)

# Adjust layout to prevent
plt.tight_layout()

plt.show()

In [ ]:
print(time_taken_tol_linearSVC)
print(accuracy_score_tol_linearSVC)
print(time_taken_tol_LogisticRegression)
print(accuracy_score_tol_LogisticRegression)

In [ ]:

#changing the penalty :

pen = ['l1', 'l2']

time_taken_pen_linearSVC = []
accuracy_score_pen_linearSVC = []
time_taken_pen_LogisticRegression = []
accuracy_score_pen_LogisticRegression = []

for i,penalty in enumerate(pen):
  model1 = LinearSVC(dual = False, penalty = penalty)
  tic = time.perf_counter()
  model1.fit(X_new_train, y_train)
  toc = time.perf_counter()
  time_taken = toc - tic
  time_taken_pen_linearSVC.append(time_taken)

  predictions =  model1.predict(X_new_test)
  acc_score = accuracy_score(y_test, predictions)
  accuracy_score_pen_linearSVC.append(acc_score)

for i,penalty in enumerate(pen):
  model2 = LogisticRegression(penalty = penalty, solver = 'liblinear')
  tic = time.perf_counter()
  model2.fit(X_new_train, y_train)
  toc = time.perf_counter()
  time_taken = toc - tic
  time_taken_pen_LogisticRegression.append(time_taken)

  predictions =  model2.predict(X_new_test)
  acc_score = accuracy_score(y_test, predictions)
  accuracy_score_pen_LogisticRegression.append(acc_score)


In [ ]:
print(time_taken_pen_linearSVC)
print(accuracy_score_pen_linearSVC)
print(time_taken_pen_LogisticRegression)
print(accuracy_score_pen_LogisticRegression)

In [ ]:

# You are allowed to import any submodules of sklearn that learn linear models e.g. sklearn.svm etc
# You are not allowed to use other libraries such as keras, tensorflow etc
# You are not allowed to use any scipy routine other than khatri_rao

# SUBMIT YOUR CODE AS A SINGLE PYTHON (.PY) FILE INSIDE A ZIP ARCHIVE
# THE NAME OF THE PYTHON FILE MUST BE submit.py

# DO NOT CHANGE THE NAME OF THE METHODS my_fit, my_map etc BELOW
# THESE WILL BE INVOKED BY THE EVALUATION SCRIPT. CHANGING THESE NAMES WILL CAUSE EVALUATION FAILURE

# You may define any new functions, variables, classes here
# For example, functions to calculate next coordinate or step length

################################
# Non Editable Region Starting #
################################
def my_fit( X_train, y_train ):
################################
#  Non Editable Region Ending  #
################################

	# Use this method to train your model using training CRPs
	# X_train has 32 columns containing the challeenge bits
	# y_train contains the responses

	# THE RETURNED MODEL SHOULD BE A SINGLE VECTOR AND A BIAS TERM
	# If you do not wish to use a bias term, set it to 0
	return w, b


################################
# Non Editable Region Starting #
################################
def my_map( X ):
################################
#  Non Editable Region Ending  #
################################

	# Use this method to create features.
	# It is likely that my_fit will internally call my_map to create features for train points

	return feat
